## intro

뇌의 작동 원리는 artificial neural networks (ANN)에 영감을 주었습니다. 하지만 비행기가 새에서 영감을 얻었다 하더라도 날개를 펄럭이진 않듯이, ANN도 점차 생물학적 영감으로부터 멀어지가 되었습니다. 몇몇 연구자들은 이제 생물학적 은유와는 거리를 두어야 한다고 주장합니다. (neuron이란 용어 대신 unit이라는 용어를 사용하는 등의 방법을 통해서)

ANN은 딥러닝 기술의 핵심입니다. 이는 다양한 목적으로 이용할 수 있고, 강력하며, 확정성이 뛰어나서 크고 복잡한 머신러닝 업무를 수행하는데 이상적입니다.

우선 간단한 형태의 ANN을 살펴본 후, MLP(Multi-Layer-Perceptron)을 살펴본 후 MNIST 데이터셋에 적용해봅시다.


## From Biological to Artificial Neurons

최초의 NN은 1943년경에 소개되었습니다. 60년대까지 승승장구하던 ANN은 이후 그 한계가 드러나 암흑기에 접어들었습니다. 80년대 초반 새로운 네트워크 구조가 발명되면서 부활하는 듯 했으나 90년대에는 ANN의 강력한 경쟁자인 SVM이 더 좋은 결과를 보여주며 더 탄탄한 이론적 근거를 제공하는 것으로 받아들여졌습니다. 하지만 이제 우리는 ANN의 부활을 다시 보고 있습니다. 이 움직임은 이전과는 다르게 쉽게 사그라들지 않을 것으로 보입니다. 그 이유는

    * training할  수 있는 데이터 규모가 방대하여 ML보다 더 좋은 성능을 보여줄 것으로 기대
    *  computing power의 비약적인 증대
    *  training algorithm의 개선
    * 몇 ANN의 이론적인 한계가 실제 적용시에는 잘 발생하지 않는 것으로 나타남 (local optima에 빠질 것으로 예상했으나, 극히 드물게 발생)
    * ANN은 이미 널리 활용, 선순환에 접어들었음

## The Perceptron

* Perceptron은 가장 간단한 ANN 구조. 1957년에 고안되었음
* Linear Threshold Unit (LTU) 구조에 기반을 두고 있음 (숫자 인풋과 숫자 아웃풋, 각 인풋에 w 곱한 후 특정한 function에 통과시켜서 output 도출)
* single LTU는 simple linear classification에 활용될 수 있음
* Perceptron은 single layer의 LTU로 구성되어 있음.
* Perceptron은 어떻게 train되는가? : Hebb's rule
    * "Cells that fire toghther, wire together"
    * 잘못된 Output으로 이어지는 input과의 연결은 약화시키고, 올바른 결정에 기여하는 input과의 연결은 강화
        * Wij = Wij + (LearningRate)x(predictedY-Y)*Xi
* Perceptron의 decision boundary는 linear하므로 perceptron은 복잡한 패턴을 분류해낼 수 없다. (Logistic Regression과 동일)
* Target이 선형으로 분류 가능한 형태로 분포해있다면 분류하는 것이 가능


#### (실습) sklearn의 Perceptron 클래스를 통한 single LTU network의 구성

In [3]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris=load_iris()
X=iris.data[:,(2,3)] # petal length, petal width 변수만 가지고 옴
y=(iris.target==0).astype(np.int) # setosa인지의 YN여부를 target으로 설정
per_clf=Perceptron(random_state=42)
per_clf.fit(X,y)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=42, shuffle=True,
      verbose=0, warm_start=False)

In [8]:
per_clf.predict([[2, 0.5]])

array([1])

* Stochastic Gradient Descent와 매우 닮은 형태라는 것을 알 수 있다.
* 실제로 Sci-kit learn의 Perceptron 클래스는 SGDClassifier에 아래 hyperparameter를 준 것과 동일하다
    * loss="preceptron"
    * learning_rate="constant"
    * eta=0.1 (learning rate)
    * penalty=None (No regularization)
<br><br>
* Perceptron은 Logistic Regression와 같이 각 class에 속할 확률을 보여주지는 않는다.
* 이 점이 Logistic Regression이 선호되는 중요한 이유이다.

#### NN의 암흑기와 Multiple Perceptron으로 인한 부활
* Marvin Minsky와 Seymour Papert는 Perceptron이 XOR 등의 비선형 문제를 해결하지 못한다고 지적
* 선형 알고리즘들은 모두 동일한 문제 갖고 있으나, NN에 대한 기대가 컸던 만큼 실망도 컸음.
* 이후connectionism에 대한 연구 자체가 중단됨
<br><br>
* 하지만 이후 multiple Perceptron을 Stacking하면 해당 문제를 해결 할 수 있다고 밝혀짐
* 해당 ANN의 이름은 MLP (Multi-Layer-Perceptron)으로 명명됨

## Multi-Layer-Perceptron and Backpropagation

* MLP는 하나의 input layer / 1개 이상의 LTU layer (Hidden Layer) / 최종 LTU Layer (Output layer)로 구성되어 있음
* ANN이 2개 이상의 hidden layer로 구성되어 있을때, 이를 Deep Neural Network(DNN)이라고 명명함
<br><br>
__backpropagation__
<br><br>
* 수년간 연구자들이 MLP를 학습시키는 방법을 찾으려 시도함
* 1986년 D.E.Rumelhartrk backpropagation을 소개하는 논문을 발표함
* 'Gradient Descent using revere-mode autodiff' 라고 설명할 수 있음
<br><br>
__학습단계__
<br><br>
1. forward pass : input이 network에 투입되면 모든 뉴런에 대한 output을 계산함 
2. 이후 output과 실제 y간의 오차를 계산한다
3. 최종 hidden layer가 오차에 어느 정도로 기여했는지 계산한다.
4. 최종 hidden layer의 오차 기여 정도를 토대로 그 이전 hidden layer의 오차 기여도를 계산한다.
5. weight를 update한다 (gradient Descent)
<br><br>
__Activation Function__
<br><br>
* backpropagation이 잘 작동할 수 있도록, step function을 logistic function으로 변경함 (logit=1/(1+exp(z))
* step 없이 계수와 w의 합만을 이용할 경우, flat한 선형이므로 gradient를 계산할 수 없음
* logit 함수 外 많이 쓰이는 함수로는 tahn과 ReLU 함수가 있음 (2시그마(2z)-1)
    * Tahn : hyperbolic Tangent Function, logistic과 같이 S모양, 연속형, 미분 가능하지만 -1~1의 범위이다. converging이 빨라지는 효과가 있다.
    * ReLU : max(0,z), 연속형이지만 0에서는 미분이 불가하다. (slope이 들쭉날쭉하게 계산되어 Gradient Descent가 튀게 된다)<br>하지만 실전에서는 매우 잘 작동하며 계산속도가 빨라지는 효과가 있다.<br>특히 최대값 제한이 없는 함수이기에 Gradient Descent를 활용하면서 생기는 문제를 해소해준다. (Vanishing Gradient)


* MLP는 주로 classification에 활용된다
* Multi-class인 경우 최종 output-layer가 __softmax function__으로 대체된다
    * softmax : 각 neuron의 아웃풋은 각 클래스에 소속될 확률

## Traning an MLP with Tesorflow's High-Level API

* tensorflow를 활용, MLP를 구현하는 가장 간단한 방법은 High-Level API인 TF.learn을 활용하는 것
* sci-kit의 API와 유사,
* DNNClassifier를 활용하면 hidden layer의 layer수, Node수, Activation Function 등을 자유자재로 활용할 수 있음

In [1]:
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [16]:
X_train=mnist.train.images
X_test=mnist.test.images
y_train=mnist.train.labels.astype("int")
y_test=mnist.test.labels.astype("int")

In [19]:
import tensorflow as tf

config=tf.contrib.learn.RunConfig(tf_random_seed=42)

feature_cols=tf.contrib.learn.infer_real_valued_columns_from_input(X_train) # input 데이터를 tensorflow에서 처리 가능한형태로 가공하는 과정
dnn_clf=tf.contrib.learn.DNNClassifier(hidden_units=[300,100],
                                                               n_classes=10,
                                                               feature_columns=feature_cols,
                                                               config=config)
dnn_clf.fit(X_train,y_train,batch_size=50,steps=40000)

INFO:tensorflow:Using config: {'_model_dir': None, '_num_worker_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001DAF466AEF0>, '_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 42, '_task_id': 0, '_task_type': None, '_environment': 'local', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_num_ps_replicas': 0, '_master': '', '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_is_chief': True, '_evaluation_master': ''}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Argume

C:\Users\RyanLee\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\RyanLee\AppData\Local\Temp\tmp6v2_4vb4\model.ckpt.
INFO:tensorflow:loss = 2.36404, step = 1
INFO:tensorflow:global_step/sec: 201.483
INFO:tensorflow:loss = 0.311432, step = 101 (0.520 sec)
INFO:tensorflow:global_step/sec: 184.693
INFO:tensorflow:loss = 0.265409, step = 201 (0.519 sec)
INFO:tensorflow:global_step/sec: 213.107
INFO:tensorflow:loss = 0.408733, step = 301 (0.468 sec)
INFO:tensorflow:global_step/sec: 208.213
INFO:tensorflow:loss = 0.244357, step = 401 (0.484 sec)
INFO:tensorflow:global_step/sec: 211.301
INFO:tensorflow:loss = 0.238858, step = 501 (0.469 

INFO:tensorflow:global_step/sec: 214.022
INFO:tensorflow:loss = 0.00680886, step = 7701 (0.470 sec)
INFO:tensorflow:global_step/sec: 181.996
INFO:tensorflow:loss = 0.00376618, step = 7801 (0.549 sec)
INFO:tensorflow:global_step/sec: 202.711
INFO:tensorflow:loss = 0.0131479, step = 7901 (0.491 sec)
INFO:tensorflow:global_step/sec: 223.62
INFO:tensorflow:loss = 0.00631671, step = 8001 (0.447 sec)
INFO:tensorflow:global_step/sec: 184.351
INFO:tensorflow:loss = 0.0245089, step = 8101 (0.541 sec)
INFO:tensorflow:global_step/sec: 220.163
INFO:tensorflow:loss = 0.0208505, step = 8201 (0.455 sec)
INFO:tensorflow:global_step/sec: 222.621
INFO:tensorflow:loss = 0.04197, step = 8301 (0.452 sec)
INFO:tensorflow:global_step/sec: 216.813
INFO:tensorflow:loss = 0.0136715, step = 8401 (0.458 sec)
INFO:tensorflow:global_step/sec: 188.534
INFO:tensorflow:loss = 0.0110502, step = 8501 (0.532 sec)
INFO:tensorflow:global_step/sec: 203.955
INFO:tensorflow:loss = 0.00604934, step = 8601 (0.488 sec)
INFO:tens

INFO:tensorflow:global_step/sec: 215.875
INFO:tensorflow:loss = 0.000447847, step = 15901 (0.462 sec)
INFO:tensorflow:global_step/sec: 225.643
INFO:tensorflow:loss = 0.00656253, step = 16001 (0.444 sec)
INFO:tensorflow:global_step/sec: 199.07
INFO:tensorflow:loss = 0.0043496, step = 16101 (0.504 sec)
INFO:tensorflow:global_step/sec: 169.328
INFO:tensorflow:loss = 7.70298e-05, step = 16201 (0.589 sec)
INFO:tensorflow:global_step/sec: 207.347
INFO:tensorflow:loss = 0.00260926, step = 16301 (0.482 sec)
INFO:tensorflow:global_step/sec: 224.122
INFO:tensorflow:loss = 0.00155471, step = 16401 (0.446 sec)
INFO:tensorflow:global_step/sec: 223.119
INFO:tensorflow:loss = 0.0015937, step = 16501 (0.448 sec)
INFO:tensorflow:global_step/sec: 223.619
INFO:tensorflow:loss = 0.00472044, step = 16601 (0.446 sec)
INFO:tensorflow:global_step/sec: 222.125
INFO:tensorflow:loss = 0.00249327, step = 16701 (0.451 sec)
INFO:tensorflow:global_step/sec: 208.213
INFO:tensorflow:loss = 0.00237052, step = 16801 (0.

INFO:tensorflow:loss = 0.00113664, step = 24001 (0.454 sec)
INFO:tensorflow:global_step/sec: 220.648
INFO:tensorflow:loss = 0.000741892, step = 24101 (0.453 sec)
INFO:tensorflow:global_step/sec: 215.411
INFO:tensorflow:loss = 0.000928554, step = 24201 (0.464 sec)
INFO:tensorflow:global_step/sec: 203.124
INFO:tensorflow:loss = 0.000218442, step = 24301 (0.493 sec)
INFO:tensorflow:global_step/sec: 198.674
INFO:tensorflow:loss = 0.00173174, step = 24401 (0.502 sec)
INFO:tensorflow:global_step/sec: 219.195
INFO:tensorflow:loss = 0.000569286, step = 24501 (0.456 sec)
INFO:tensorflow:global_step/sec: 177.78
INFO:tensorflow:loss = 0.000322807, step = 24601 (0.562 sec)
INFO:tensorflow:global_step/sec: 224.12
INFO:tensorflow:loss = 0.00111121, step = 24701 (0.445 sec)
INFO:tensorflow:global_step/sec: 224.627
INFO:tensorflow:loss = 0.00132868, step = 24801 (0.457 sec)
INFO:tensorflow:global_step/sec: 216.813
INFO:tensorflow:loss = 0.00235335, step = 24901 (0.452 sec)
INFO:tensorflow:global_step/

INFO:tensorflow:loss = 0.000425281, step = 32101 (0.457 sec)
INFO:tensorflow:global_step/sec: 179.701
INFO:tensorflow:loss = 0.00102074, step = 32201 (0.559 sec)
INFO:tensorflow:global_step/sec: 202.712
INFO:tensorflow:loss = 0.00041044, step = 32301 (0.491 sec)
INFO:tensorflow:global_step/sec: 197.103
INFO:tensorflow:loss = 0.000104019, step = 32401 (0.506 sec)
INFO:tensorflow:global_step/sec: 221.631
INFO:tensorflow:loss = 0.000511087, step = 32501 (0.452 sec)
INFO:tensorflow:global_step/sec: 221.629
INFO:tensorflow:loss = 0.000111053, step = 32601 (0.450 sec)
INFO:tensorflow:global_step/sec: 217.763
INFO:tensorflow:loss = 0.00126878, step = 32701 (0.462 sec)
INFO:tensorflow:global_step/sec: 218.715
INFO:tensorflow:loss = 0.000895224, step = 32801 (0.456 sec)
INFO:tensorflow:global_step/sec: 168.755
INFO:tensorflow:loss = 0.000406547, step = 32901 (0.594 sec)
INFO:tensorflow:global_step/sec: 179.378
INFO:tensorflow:loss = 0.000497633, step = 33001 (0.557 sec)
INFO:tensorflow:global_s

DNNClassifier(params={'input_layer_min_slice_size': None, 'embedding_lr_multipliers': None, 'optimizer': None, 'gradient_clip_norm': None, 'dropout': None, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x000001DAF4667BE0>, 'activation_fn': <function relu at 0x000001DAEFAD2378>, 'hidden_units': [300, 100], 'feature_columns': (_RealValuedColumn(column_name='', dimension=784, default_value=None, dtype=tf.float32, normalizer=None),)})

In [26]:
from sklearn.metrics import accuracy_score

y_pred=list(dnn_clf.predict(X_test)) # 결과를 List 형태로 만들어야 accuracy_score의 계산이 가능하다
accuracy_score(y_test,y_pred)

Instructions for updating:
Please switch to predict_classes, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


C:\Users\RyanLee\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Restoring parameters from C:\Users\RyanLee\AppData\Local\Temp\tmp6v2_4vb4\model.ckpt-40000


0.98209999999999997

In [30]:
dnn_clf.evaluate(X_test,y_test)

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


C:\Users\RyanLee\Anaconda3\lib\site-packages\tensorflow\python\util\deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-08-15-07:55:15
INFO:tensorflow:Restoring parameters from C:\Users\RyanLee\AppData\Local\Temp\tmp6v2_4vb4\model.ckpt-40000
INFO:tensorflow:Finished evaluation at 2017-08-15-07:55:16
INFO:tensorflow:Saving dict for global step 40000: accuracy = 0.9821, global_step = 40000, loss = 0.0719601


{'accuracy': 0.98210001, 'global_step': 40000, 'loss': 0.071960069}

*  DNNClassifier는 ReLU / softmax / cross entropy 기준으로 평가

## Training a DNN Using Plain TensorFlow

### Construction Phase

In [6]:
import tensorflow as tf

n_inputs=28*28
n_hidden1=300
n_hidden2=100
n_outputs=10

##### Construction Phase

In [14]:
X=tf.placeholder(tf.float32,shape=(None,n_inputs), name="X")
y=tf.placeholder(tf.int64, shape=(None), name="y")

In [16]:
from tensorflow.contrib.layers import fully_connected

with tf.name_scope("dnn"):
    hidden1=fully_connected(X,n_hidden1,scope="hidden1",reuse=True)
    hidden2=fully_connected(hidden1,n_hidden2,scope="hidden2",reuse=True)
    logits=fully_connected(hidden2,n_outputs,scope="outputs",activation_fn=None,reuse=True)

In [17]:
with tf.name_scope("loss"):
    xentropy=tf.nn.sparse_softmax_cross_entropy_with_logits(
                        labels=y, logits=logits)
    loss=tf.reduce_mean(xentropy,name="loss")

In [18]:
learning_rate=0.01

with tf.name_scope("train"):
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    training_op=optimizer.minimize(loss)

In [19]:
with tf.name_scope("eval"):
    correct=tf.nn.in_top_k(logits,y,1) # Logit으로 선정된 1순위가 y와 일치하는지 여부를 boolean으로 표기
    accuracy=tf.reduce_mean(tf.cast(correct,tf.float32))

In [20]:
init=tf.global_variables_initializer()
saver=tf.train.Saver()

##### Execution Phase

In [21]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("/tmp/data")

Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


In [22]:
n_epochs=400
batch_size=50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples//batch_size):
            X_batch, y_batch=mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X:X_batch,y:y_batch})
        acc_train=accuracy.eval(feed_dict={X:X_batch,y:y_batch})
        acc_test=accuracy.eval(feed_dict={X:mnist.test.images,y:mnist.test.labels})
        print(epoch,"Train accu : ",acc_train,"Test accu : ",acc_test)
        
    save_path=saver.save(sess,"C:/Users/RyanLee/Documents/hands-on/mnist_dnn.ckpt")

0 Train accu :  0.9 Test accu :  0.9003
1 Train accu :  0.94 Test accu :  0.9171
2 Train accu :  0.94 Test accu :  0.9293
3 Train accu :  0.98 Test accu :  0.9355
4 Train accu :  0.94 Test accu :  0.9421
5 Train accu :  0.94 Test accu :  0.9439
6 Train accu :  0.98 Test accu :  0.9483
7 Train accu :  0.96 Test accu :  0.9508
8 Train accu :  0.98 Test accu :  0.9547
9 Train accu :  0.94 Test accu :  0.9565
10 Train accu :  0.94 Test accu :  0.9585
11 Train accu :  1.0 Test accu :  0.9607
12 Train accu :  0.98 Test accu :  0.9628
13 Train accu :  1.0 Test accu :  0.9641
14 Train accu :  1.0 Test accu :  0.965
15 Train accu :  0.98 Test accu :  0.967
16 Train accu :  0.98 Test accu :  0.9681
17 Train accu :  1.0 Test accu :  0.9697
18 Train accu :  1.0 Test accu :  0.9707
19 Train accu :  1.0 Test accu :  0.9712
20 Train accu :  0.96 Test accu :  0.972
21 Train accu :  0.96 Test accu :  0.9734
22 Train accu :  1.0 Test accu :  0.9742
23 Train accu :  0.98 Test accu :  0.9746
24 Train accu

198 Train accu :  1.0 Test accu :  0.9814
199 Train accu :  1.0 Test accu :  0.9811
200 Train accu :  1.0 Test accu :  0.9811
201 Train accu :  1.0 Test accu :  0.9807
202 Train accu :  1.0 Test accu :  0.981
203 Train accu :  1.0 Test accu :  0.9807
204 Train accu :  1.0 Test accu :  0.9813
205 Train accu :  1.0 Test accu :  0.9811
206 Train accu :  1.0 Test accu :  0.981
207 Train accu :  1.0 Test accu :  0.9813
208 Train accu :  1.0 Test accu :  0.981
209 Train accu :  1.0 Test accu :  0.981
210 Train accu :  1.0 Test accu :  0.9814
211 Train accu :  1.0 Test accu :  0.9811
212 Train accu :  1.0 Test accu :  0.9811
213 Train accu :  1.0 Test accu :  0.9812
214 Train accu :  1.0 Test accu :  0.9813
215 Train accu :  1.0 Test accu :  0.9813
216 Train accu :  1.0 Test accu :  0.9812
217 Train accu :  1.0 Test accu :  0.9811
218 Train accu :  1.0 Test accu :  0.9813
219 Train accu :  1.0 Test accu :  0.9812
220 Train accu :  1.0 Test accu :  0.981
221 Train accu :  1.0 Test accu :  0.98

395 Train accu :  1.0 Test accu :  0.9811
396 Train accu :  1.0 Test accu :  0.981
397 Train accu :  1.0 Test accu :  0.981
398 Train accu :  1.0 Test accu :  0.9809
399 Train accu :  1.0 Test accu :  0.981


In [24]:
import numpy as np

with tf.Session() as sess:
    saver.restore(sess,"C:/Users/RyanLee/Documents/hands-on/mnist_dnn.ckpt")
    X_new_scaled=mnist.test.images
    Z=logits.eval(feed_dict={X:X_new_scaled})
    y_pred=np.argmax(Z,axis=1)

INFO:tensorflow:Restoring parameters from C:/Users/RyanLee/Documents/hands-on/mnist_dnn.ckpt
